## Desarrollo de la API


In [2]:
import pandas as pd
import pyarrow as pa
import ast
import numpy as np
import csv
from textblob import TextBlob
import fastparquet as fp 
import pyarrow.parquet as pq

## Vamos a recordar que campos tienen nuestros data sets revisando el Diccionario de Datos

 El archivo `steam_games` contiene la información sobre los juegos de la plataforma Steam :
   * **genres**: es el género del item, es decir, del juego. Esta formado por una lista de uno o mas géneros por registro.
   * **url**: es la url del juego.
   * **release_date**: es la fecha de lanzamiento del item en formato 2018-01-04.
   * **tags**: es la etiqueta del contenido. Esta formado por una lista de uno o mas etiquetas por registro.
   * **reviews_url**: es la url donde se encuentra el review de ese juego.
   * **specs**: son especificaciones de cada item. Es una lista con uno o mas string con las especificaciones.
   * **price**: es el precio del item.
   * **early_access**: indica el acceso temprano con un True/False.
   * **id**: es el identificador único del contenido.
   * **año_estreno**: es el año de lanzamiento del item.
   * **publisher**: es la empresa publicadora del contenido.
   * **app_name**: es el nombre del item, es decir, del juego.
   * **title**: es el título del items
   * **developer**: desarrollador del contenido.

In [3]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\steam_games.csv"
steam_games= pd.read_csv(archivo)
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

El archivo `user_review` contiene información relacionada con las reviews que realizaron los usuarios de los juegos:
* **user_id**: es un identificador único para el usuario.
* **user_url**: es la url del perfil del usuario en streamcommunity.
* **reviews**: contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionario con el review. Cada diccionario contiene:
    *    **funny**: indica si alguien puso emoticón de gracioso al review.
    *   **posted**: es la fecha de posteo del review en formato Posted April 21, 2011.
    *   **last_edited**: es la fecha de la última edición.
    *   **item_id**: es el identificador único del item, es decir, del juego.
    *   **helpful**: es la estadística donde otros usuarios indican si fue útil la información.
    *   **recommend**: es un booleano que indica si el usuario recomienda o no el juego.
    *   **review**: es una sentencia string con los comentarios sobre el juego. "se cambió por `sentiment_analisis` agregando       0 , 1 o 2 

In [4]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_reviews_final.csv"
user_reviews_final= pd.read_csv(archivo)
user_reviews_final.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analisis'],
      dtype='object')

El archivo `user_items` contiene información con respecto al consumo de juegos de la plataforma por parte de los usuarios:
* **user_id**: contiene un identificador único del usuario.
* **items_count**: contiene un número entero que indica la cantidad de juegos que ha consumido el usuario.
* **steam_id**: es un número único para la plataforma.
* **user_url**: es la url del perfil del usuario
* **items**: contiene una lista de uno o mas diccionarios de los items que consume cada usuario. Cada diccionario tiene las siguientes claves:
    * **item_id**: es el identificados del item, es decir, del juego.
    * **item_name**: es el nombre del contenido que consume, es decir, del juego.
    * **playtime_forever**: es el tiempo acumulado que un usuario jugó.
    * **playtime_2weeks**: es el tiempo acumulado que un usuario jugó durante 2 semanas.

In [5]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_items.parquet"
user_items= pd.read_parquet(archivo)
user_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'steam_id', 'items_count', 'user_id', 'user_url'],
      dtype='object')

## Funciones a crear para consumo de la API

* **def PlayTimeGenre**( genero : str ): Debe devolver año con más horas jugadas para dicho género.
    **Ejemplo** de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

* **def UserForGenre**( genero : str ): Debe devolver al usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    **Ejemplo** de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

* **def UsersRecommend** ( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    **Ejemplo** de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" :Z}]

* **def UsersNotRecommend**( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    **Ejemplo** de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" :Z}]

* **def sentiment_analysis**( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento.
    **Ejemplo** de retorno: {Negativo = 182, Neutral = 120, Positivo = 278}

## Función `def_PlayTimeGenre`( genero : str )

( genero : str ): Debe devolver `año con más horas jugadas` para `dicho género`.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [6]:
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

In [7]:
user_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'steam_id', 'items_count', 'user_id', 'user_url'],
      dtype='object')

La información que necesitamos incluye los géneros, el año de lanzamiento y el tiempo jugado en total por cada juego. Para ello, vamos a utilizar las columnas 'id', 'genres', 'Año de estreno' extraido de 'release date' del conjunto de datos `steam_games`, y las columnas 'item_id' y 'playtime_forever' del conjunto de datos `user_items`. Vamos a concatenar los DataFrames a través de las columnas 'id' e 'item_id'.

In [8]:
# Combinar DataFrames utilizando las variables en español
tiempo_juego_por_genero = pd.merge(steam_games[['id', 'genres', 'Año_estreno']],
                               user_items[['item_id', 'playtime_forever']],
                               left_on='id', right_on='item_id', how='inner')

In [9]:
tiempo_juego_por_genero.head()

,id,genres,Año_estreno,item_id,playtime_forever
0,282010,Action,1997,282010,5
1,282010,Action,1997,282010,0
2,282010,Action,1997,282010,0
3,282010,Action,1997,282010,0
4,282010,Action,1997,282010,13


In [11]:
##Agrupar la columna 'playtime_forever' por cada 'id' para obtener el tiempo total jugado por cada juego
tiempo_juego_por_genero = tiempo_juego_por_genero.groupby(['genres', 'Año_estreno', 'id'])['playtime_forever'].sum().reset_index()

In [12]:
tiempo_juego_por_genero.head()

,genres,Año_estreno,id,playtime_forever
0,Action,1983,227380,3473
1,Action,1984,240340,384
2,Action,1988,308000,16001
3,Action,1989,302330,607
4,Action,1990,9180,18266


In [13]:
#Agrupamos por 'genres' y 'Año_estreno', sumando las horas jugadas para cada combinación.
#Agrupamos por genero y año de lanzamiento
tiempo_juego_por_genero = tiempo_juego_por_genero.groupby(['Año_estreno', 'genres'])['playtime_forever'].sum().reset_index()

In [14]:
tiempo_juego_por_genero.head()

,Año_estreno,genres,playtime_forever
0,1983,Action,3473
1,1983,Adventure,3473
2,1983,Casual,3473
3,1984,Action,384
4,1984,Adventure,384


El resultado es un DataFrame que contiene la información de cuántas horas se han jugado por género y año de lanzamiento. Convertiremos este DataFrame a un archivo CSV para alimentar la API.

In [15]:
# Guardar el DataFrame en un archivo CSV
tiempo_juego_por_genero.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\PlayTimeGenre.csv', index=False)

Creación de la función para consultar el año con más horas jugadas por género

In [20]:
def PlayTimeGenre(genero):
    """
    Función que toma como entrada un género y devuelve el año con la máxima cantidad de horas jugadas para ese género.

    Parámetros:
    - genero (str): Género para el cual se busca el año con más horas jugadas.

    Ejemplo de retorno:
    - Para un género existente: 2013
    - Para un género no encontrado: "No se encontraron datos para el género 'Género X'"
    """
    # Filtra el DataFrame para el género especificado

    df_genero = tiempo_juego_por_genero[tiempo_juego_por_genero['genres'] == genero]

    # Verifica si el género está presente en el DataFrame
    if not df_genero.empty:
        # Encuentra el año con la máxima cantidad de horas jugadas
        anio_mas_jugado = df_genero.loc[df_genero['playtime_forever'].idxmax()]['Año_estreno']
        return f'Año de lanzamiento con más horas jugadas para el género {genero} : {int(anio_mas_jugado)}'
    else:
        return f"No se encontraron datos para el género '{genero}'"

In [21]:
PlayTimeGenre('Adventure')

'Año de lanzamiento con más horas jugadas para el género Adventure : 2011'

## Función `def_UserForGenre`( genero : str )

**def UserForGenre**( genero : str ): Debe devolver al `usuario que acumula más horas jugadas para el género dado` y una `lista de la acumulación de horas jugadas por año`.

**Ejemplo** de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

La información necesaria para esta consulta se encuentra en los dataframes `steam_games` y `user_items`. Del dataframe `steam_games`, requerimos las columnas 'genres' y 'Año_estreno', mientras que del dataframe `user_items` necesitamos los datos de 'playtime_forever' y 'user_id'. Para lograr esto, procederemos a concatenar los dataframes utilizando la columna 'item_id'.

In [25]:
#Lectura de los Archivos CSV
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\steam_games.csv"
steam_games= pd.read_csv(archivo)
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

In [24]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_items.parquet"
user_items= pd.read_parquet(archivo)
user_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'steam_id', 'items_count', 'user_id', 'user_url'],
      dtype='object')

In [26]:
# Unimos los DataFrames usando las columnas 'id' en steam_games y 'item_id' en user_items
df_UserForGenre = pd.merge(user_items, steam_games, left_on='item_id', right_on='id', how='inner')

# Seleccionar las columnas requeridas
df_UserForGenre = df_UserForGenre[['genres', 'Año_estreno', 'playtime_forever', 'user_id']]

df_UserForGenre.head()

,genres,Año_estreno,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,0,js41637
2,Action,2000,0,Riot-Punch
3,Action,2000,93,doctr
4,Action,2000,108,corrupted_soul


In [27]:
# Buscamos el user_id con la mayor cantidad de horas jugadas por género
jugador_mas_horas = df_UserForGenre.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Buscamos el índice del user_id con la mayor cantidad de horas jugadas por género
indice = jugador_mas_horas.groupby('genres')['playtime_forever'].idxmax()

# Filtrar el DataFrame original usando los índices encontrados
jugador_mas_horas = jugador_mas_horas.loc[indice]

In [28]:
jugador_mas_horas.head()

,genres,user_id,playtime_forever
50657,Action,Sp3ctre,1699307
112560,Adventure,REBAS_AS_F-T,2191551
132829,Animation &amp; Modeling,ScottyG555,168314
134877,Audio Production,Lickidactyl,109916
175051,Casual,REBAS_AS_F-T,1224933


In [29]:
#Concatenamos los DataFrames jugador_mas_horas y df_UserForGenre para retener únicamente al jugador que ha acumulado la mayor cantidad de horas por género del DataFrame df_UserForGenre. Además, conservamos la información sobre la cantidad de horas jugadas por año de lanzamiento.

df_UserForGenre = pd.merge(df_UserForGenre, jugador_mas_horas[['genres', 'user_id']], on=['genres','user_id'])
df_UserForGenre.head()

,genres,Año_estreno,playtime_forever,user_id
0,Action,2000,70644,Sp3ctre
1,Action,1999,44,Sp3ctre
2,Action,1999,0,Sp3ctre
3,Action,1998,0,Sp3ctre
4,Action,2001,13,Sp3ctre


In [30]:
#Filtramos los valores para eliminar las entradas donde la cantidad de horas jugadas (playtime_forever) es igual a 0.
df_UserForGenre = df_UserForGenre[df_UserForGenre['playtime_forever']>0]
df_UserForGenre.head()

,genres,Año_estreno,playtime_forever,user_id
0,Action,2000,70644,Sp3ctre
1,Action,1999,44,Sp3ctre
4,Action,2001,13,Sp3ctre
5,Action,2010,249,Sp3ctre
6,Action,2004,122426,Sp3ctre


In [31]:
#Agrupamos por Año_estreno
df_UserForGenre = df_UserForGenre.groupby(['genres', 'Año_estreno','user_id'])['playtime_forever'].sum().reset_index()
df_UserForGenre.head()

,genres,Año_estreno,user_id,playtime_forever
0,Action,1995,Sp3ctre,217
1,Action,1999,Sp3ctre,44
2,Action,2000,Sp3ctre,70644
3,Action,2001,Sp3ctre,13
4,Action,2002,Sp3ctre,238


In [32]:
#Convertimos a CSV y guardamos en la carpeta Data_Sets_API
df_UserForGenre.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\UserForGenre.csv', index=False)

Creación de la función para ver al `usuario que acumula más horas jugadas para el género dado` y una `lista de la acumulación de horas jugadas por año`.

In [48]:
def UserForGenre(genre):
    # Filtrar el DataFrame por el género dado
    genero = df_UserForGenre[df_UserForGenre['genres'] == genre]

    # Crear una lista de diccionarios para las horas jugadas por año
    horas_anio = [{'Año': year, 'Horas': playtime} for year, playtime in genero[['Año_estreno', 'playtime_forever']].values]

    # Crear el diccionario de salida
    resultado = {
        'Usuario con más horas jugadas para género '+ genre: genero.iloc[0]['user_id'],
        "Horas jugadas": horas_anio
    }

    return resultado

In [49]:
UserForGenre('Action')

{'Usuario con más horas jugadas para género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': '1995', 'Horas': 217},
  {'Año': '1999', 'Horas': 44},
  {'Año': '2000', 'Horas': 70644},
  {'Año': '2001', 'Horas': 13},
  {'Año': '2002', 'Horas': 238},
  {'Año': '2003', 'Horas': 7673},
  {'Año': '2004', 'Horas': 127411},
  {'Año': '2005', 'Horas': 21339},
  {'Año': '2006', 'Horas': 896},
  {'Año': '2007', 'Horas': 112784},
  {'Año': '2008', 'Horas': 224},
  {'Año': '2009', 'Horas': 108326},
  {'Año': '2010', 'Horas': 78083},
  {'Año': '2011', 'Horas': 154896},
  {'Año': '2012', 'Horas': 378296},
  {'Año': '2013', 'Horas': 120461},
  {'Año': '2014', 'Horas': 130691},
  {'Año': '2015', 'Horas': 312511},
  {'Año': '2016', 'Horas': 29576},
  {'Año': '2017', 'Horas': 43327},
  {'Año': 'sin dato', 'Horas': 1657}]}

## Función `def_UsersRecommend`( año : int )

* **def Recomendación de usuarios** ( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    **Ejemplo** de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" :Z}]

La información necesaria para esta consulta se encuentra en los dataframes `steam_games` y `user_reviews_final`.

In [50]:
#Lectura de los Archivos CSV
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\steam_games.csv"
steam_games= pd.read_csv(archivo)
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

In [51]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_reviews_final.csv"
user_reviews_final= pd.read_csv(archivo)
user_reviews_final.columns  

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analisis'],
      dtype='object')

In [52]:
# Concatenamos los Data Frame usando las columnas id e item_id
df_UsersRecommend = pd.merge(user_reviews_final, steam_games, left_on='item_id', right_on='id', how='inner')
df_UsersRecommend.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,genres,...,reviews_url,specs,price,early_access,id,Año_estreno,publisher,app_name,title,developer
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,2011-11-05,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,2015-03-30,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,2013-07-12,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,2015-08-13,NaN,1250,No ratings yet,True,1,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,2014-04-05,NaN,1250,No ratings yet,True,1,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive


Ajustamos la columna Posted paso que no se hizo en el ETL, esta fecha es la fecha en que se realizo la reseña

In [56]:
#Convertimos a fecha la columna posted
df_UsersRecommend['posted'] = pd.to_datetime(df_UsersRecommend['posted'], errors='coerce')
#Extraemos el año y creamos otra columna llamada posted_year
df_UsersRecommend['posted_year'] = df_UsersRecommend['posted'].dt.year
#Borramos la columna posted
df_UsersRecommend=df_UsersRecommend.drop('posted',axis=1)

In [57]:
df_UsersRecommend.head()

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,sentiment_analisis,genres,url,...,specs,price,early_access,id,Año_estreno,publisher,app_name,title,developer,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2011.0
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2015.0
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2013.0
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,NaN,1250,No ratings yet,True,1,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2015.0
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,NaN,1250,No ratings yet,True,1,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2014.0


In [59]:
# Seleccionar las columnas necesarias
selected_columns = ['app_name', 'posted_year', 'recommend', 'sentiment_analisis']
df_UsersRecommend = df_UsersRecommend[selected_columns]
df_UsersRecommend.head()

,app_name,posted_year,recommend,sentiment_analisis
0,Killing Floor,2011.0,True,2
1,Killing Floor,2015.0,True,2
2,Killing Floor,2013.0,True,2
3,Killing Floor,2015.0,True,1
4,Killing Floor,2014.0,True,1


In [60]:
#Filtramos los comentarios positivos y neutrales
df_UsersRecommend =df_UsersRecommend[df_UsersRecommend['sentiment_analisis']!=0]
df_UsersRecommend.head()

,app_name,posted_year,recommend,sentiment_analisis
0,Killing Floor,2011.0,True,2
1,Killing Floor,2015.0,True,2
2,Killing Floor,2013.0,True,2
3,Killing Floor,2015.0,True,1
4,Killing Floor,2014.0,True,1


In [61]:
#Agrupamos por el nombre del juego y año de recpmendacion: por app_name , posted_year y sumamos la cantidad de recomendaciones
df_UsersRecommend = df_UsersRecommend.groupby(['app_name', 'posted_year']).agg({'recommend': 'sum'}).reset_index()
df_UsersRecommend = df_UsersRecommend.sort_values('recommend',ascending=False)
df_UsersRecommend.head()

,app_name,posted_year,recommend
3255,Team Fortress 2,2014.0,2852
3677,Unturned,2014.0,2515
2728,Rust,2014.0,2100
3254,Team Fortress 2,2013.0,1474
1409,Garry's Mod,2014.0,1356


In [62]:
#Convertimos a CSV y guardamos en la carpeta Data_Sets_API
df_UsersRecommend.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\UsersRecommend.csv', index=False)

Creación de la funcion UsersRecommend para ver el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [84]:
def UsersRecommend(anio):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    
    Args:
    - anio (int): El año para el cual se desean obtener las recomendaciones.

    Returns:
    - dict: Un diccionario que contiene el top 3 de juegos recomendados en el formato:
        {"Puesto 1": "Nombre del Juego1", "Puesto 2": "Nombre del Juego2", "Puesto 3": "Nombre del Juego3"}
        En caso de no haber recomendaciones para el año especificado, devuelve un mensaje indicando esto.
    """

    # Filtrar las recomendaciones para el año dado y recomendaciones positivas/neutrales
    recomendaciones = df_UsersRecommend[df_UsersRecommend['posted_year'] == anio]

    # Verificar si hay revisiones para el año dado
    if recomendaciones.empty:
        return f"No hay recomendaciones para el año {anio}"

    # Ordenar en orden descendente por la cantidad de recomendaciones
    recomendaciones = recomendaciones.sort_values('recommend', ascending=False)

    # Crear una única línea de resultado
    resultado = {
        "Puesto 1": recomendaciones.iloc[0]['app_name'],
        "Puesto 2": recomendaciones.iloc[1]['app_name'],
        "Puesto 3": recomendaciones.iloc[2]['app_name']
    }

    return resultado

## Función def `UsersNotRecommend`( año : int )

Devuelve el `top 3 de juegos MENOS recomendados por usuarios para el año dado`. (reviews.recommend = False y comentarios negativos)
    **Ejemplo** de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" :Z}]

La información necesaria para esta consulta se encuentra en los dataframes `steam_games` y `user_reviews_final`.

In [65]:
#Lectura de los Archivos CSV
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\steam_games.csv"
steam_games= pd.read_csv(archivo)
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

In [66]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_reviews_final.csv"
user_reviews_final= pd.read_csv(archivo)
user_reviews_final.columns  

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analisis'],
      dtype='object')

In [68]:
# Concatenamos los Data Frame usando las columnas id e item_id
df_UsersNotRecommend = pd.merge(user_reviews_final, steam_games, left_on='item_id', right_on='id', how='inner')
df_UsersNotRecommend.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,genres,...,reviews_url,specs,price,early_access,id,Año_estreno,publisher,app_name,title,developer
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,2011-11-05,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,2015-03-30,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,2013-07-12,NaN,1250,No ratings yet,True,2,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,2015-08-13,NaN,1250,No ratings yet,True,1,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,2014-04-05,NaN,1250,No ratings yet,True,1,Action,...,http://steamcommunity.com/app/1250/reviews/?br...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive


Ajustamos la columna Posted paso que no se hizo en el ETL, esta fecha es la fecha en que se realizo la reseña

In [69]:
#Convertimos a fecha la columna posted
df_UsersNotRecommend['posted'] = pd.to_datetime(df_UsersNotRecommend['posted'], errors='coerce')
#Extraemos el año y creamos otra columna llamada posted_year
df_UsersNotRecommend['posted_year'] = df_UsersNotRecommend['posted'].dt.year
#Borramos la columna posted
df_UsersNotRecommend=df_UsersNotRecommend.drop('posted',axis=1)

In [70]:
df_UsersNotRecommend.head()

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,sentiment_analisis,genres,url,...,specs,price,early_access,id,Año_estreno,publisher,app_name,title,developer,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2011.0
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2015.0
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,NaN,1250,No ratings yet,True,2,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2013.0
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,NaN,1250,No ratings yet,True,1,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2015.0
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,NaN,1250,No ratings yet,True,1,Action,http://store.steampowered.com/app/1250/Killing...,...,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,False,1250,2009,Tripwire Interactive,Killing Floor,Killing Floor,Tripwire Interactive,2014.0


In [72]:
# Seleccionar las columnas necesarias
selected_columns = ['app_name', 'posted_year', 'recommend', 'sentiment_analisis']
df_UsersNotRecommend = df_UsersNotRecommend[selected_columns]
df_UsersNotRecommend.head()

,app_name,posted_year,recommend,sentiment_analisis
0,Killing Floor,2011.0,True,2
1,Killing Floor,2015.0,True,2
2,Killing Floor,2013.0,True,2
3,Killing Floor,2015.0,True,1
4,Killing Floor,2014.0,True,1


In [74]:
#Filtramos los comentarios negativos
df_UsersNotRecommend = df_UsersNotRecommend[df_UsersNotRecommend['sentiment_analisis'] == 0]
df_UsersNotRecommend.head()

,app_name,posted_year,recommend,sentiment_analisis
12,Killing Floor,2015.0,True,0
16,Killing Floor,2013.0,True,0
21,Killing Floor,2013.0,True,0
25,Killing Floor,2013.0,True,0
29,Killing Floor,2014.0,False,0


In [75]:
#Agrupamos por el nombre del juego y año de recomendacion: por app_name , posted_year y sumamos la cantidad de recomendaciones
df_UsersNotRecommend = df_UsersNotRecommend.groupby(['app_name', 'posted_year']).agg({'recommend': 'sum'}).reset_index()
df_UsersNotRecommend = df_UsersNotRecommend.sort_values('recommend',ascending=False)
df_UsersNotRecommend.head()

,app_name,posted_year,recommend
1269,Team Fortress 2,2014.0,268
1437,Unturned,2014.0,265
1065,Rust,2014.0,240
337,DayZ,2014.0,208
291,Counter-Strike: Global Offensive,2015.0,199


In [76]:
#Convertimos a CSV y guardamos en la carpeta Data_Sets_API
df_UsersNotRecommend.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\UsersNotRecommend.csv', index=False)

Creación de la funcion UsersNotRecommend para ver top 3 de juegos **MENOS** recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [82]:
def UsersNotRecommend(anio):
    """
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
    
    Args:
    - anio (int): El año para el cual se desean obtener las recomendaciones.

    Returns:
    - dict: Un diccionario que contiene el top 3 de juegos MENOS recomendados en el formato:
        {"Puesto 1": "Nombre del Juego1", "Puesto 2": "Nombre del Juego2", "Puesto 3": "Nombre del Juego3"}
        En caso de no haber recomendaciones para el año especificado, devuelve un mensaje indicando esto.
    """

    # Filtrar las recomendaciones para el año dado y recomendaciones positivas/neutrales
    recomendaciones_2 = df_UsersNotRecommend[df_UsersNotRecommend['posted_year'] == anio]

    # Verificar si hay revisiones para el año dado
    if recomendaciones_2.empty:
        return f"No hay recomendaciones para el año {anio}"

    # Ordenar en orden descendente por la cantidad de recomendaciones
    recomendaciones_2 = recomendaciones_2.sort_values('recommend', ascending=False)

    # Crear una única línea de resultado
    resultado = {
        "Puesto 1": recomendaciones_2.iloc[0]['app_name'],
        "Puesto 2": recomendaciones_2.iloc[1]['app_name'],
        "Puesto 3": recomendaciones_2.iloc[2]['app_name']
    }

    return resultado

In [83]:
UsersNotRecommend(2011)

{'Puesto 1': 'Terraria',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': 'Spiral Knights'}

## Función def `sentiment_analysis`( año : int )

**def sentiment_analysis**( año: int ) : Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento.
                            Ejemplo de retorno: {Negativo = 182, Neutral = 120, Positivo = 278}

La información necesaria para esta consulta se encuentra en los dataframes `steam_games` y `user_reviews_final`.

In [3]:
import pandas as pd
import csv

In [4]:
#Lectura de los Archivos CSV
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\steam_games.csv"
steam_games= pd.read_csv(archivo)
steam_games.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url', 'specs',
       'price', 'early_access', 'id', 'Año_estreno', 'publisher', 'app_name',
       'title', 'developer'],
      dtype='object')

In [5]:
archivo = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\user_reviews_final.csv"
user_reviews_final= pd.read_csv(archivo)
user_reviews_final.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analisis'],
      dtype='object')

In [6]:
#Se extraen columnas del dataframe "steam_games" 
anio_estreno=steam_games[["id","Año_estreno"]]
#Se renombran las columnas necesarias
anio_estreno= anio_estreno.rename(columns={"id":"item_id", "Año_estreno":"release_anio"})
anio_estreno.head()

,item_id,release_anio
0,761140,2018
1,761140,2018
2,761140,2018
3,761140,2018
4,761140,2018


In [7]:
#se eliminan duplicados para realizar el merge "concatenado"
anio_estreno= anio_estreno.drop_duplicates()
anio_estreno

,item_id,release_anio
0,761140,2018
5,643980,2018
9,670290,2017
14,767400,2017
17,772540,2018
...,...,...
71535,745400,2018
71539,773640,2018
71543,733530,2018
71546,610660,2018


In [8]:
anio_estreno ["release_anio"]= anio_estreno["release_anio"].astype (int, errors="ignore")
anio_estreno.dtypes

item_id          int64
release_anio    object
dtype: object

In [9]:
#se le anexa el año de estreno al dataframe "user_reviews"
sentiment_analysis=user_reviews_final.merge(anio_estreno, on="item_id")
sentiment_analysis.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,2011-11-05,NaN,1250,No ratings yet,True,2,2009
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,2015-03-30,NaN,1250,No ratings yet,True,2,2009
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,2013-07-12,NaN,1250,No ratings yet,True,2,2009
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,2015-08-13,NaN,1250,No ratings yet,True,1,2009
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,2014-04-05,NaN,1250,No ratings yet,True,1,2009


In [92]:
#Convertimos a CSV y guardamos en la carpeta Data_Sets_API
sentiment_analysis.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\sentiment_Analysis.csv', index=False)

In [12]:
#Inicialmente al ejecutar la función sentiment_analysis me salia el error TypeError: 'function' object is not subscriptable, para solucionarlo Cargamo nuevamente el data fame y le cambio el nombre 
df = r"C:\Users\ayrc2\Documentos\Proyecto Individual DPT03\PI_ML_OPS_Steam_Games\Data_sets_API\sentiment_Analysis.csv"
sentimiento_analysis= pd.read_csv(df)
sentimiento_analysis.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,2011-11-05,NaN,1250,No ratings yet,True,2,2009
1,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,2015-03-30,NaN,1250,No ratings yet,True,2,2009
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,NaN,2013-07-12,NaN,1250,No ratings yet,True,2,2009
3,diego9031,http://steamcommunity.com/id/diego9031,NaN,2015-08-13,NaN,1250,No ratings yet,True,1,2009
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,NaN,2014-04-05,NaN,1250,No ratings yet,True,1,2009


In [15]:
sentimiento_analysis.to_csv('..\\PI_ML_OPS_Steam_Games\\Data_sets_API\\sentimiento_analysis.csv', index=False)

Creación de la funcion sentiment_analisis Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentran categorizados con un análisis de sentimiento

In [13]:
def sentiment_analysis(anio):
    #Se filtran las reviews por año y las igualo al año que se ingresa en la consulta transformandolo en string 
    reviews_por_anio= sentimiento_analysis[sentimiento_analysis["release_anio"]== str(anio)]
    
    #Se inicia una lista vacia por cada sentimiento para ir contandolos 
    Negativos = 0
    Neutral = 0
    Positivos = 0
    
    #Se itera sobre las filas de reviews_por_anio y se distibuyen los datos segun la columna "sentiment_analysis"
    for i in reviews_por_anio["sentiment_analisis"]:
        if i == 0:
            Negativos += 1
        elif i == 1:
            Neutral += 1 
        elif i == 2:
            Positivos += 1

    count_sentiment ={"Negative": Negativos , "Neutral" : Neutral, "Positive": Positivos}
    
    return count_sentiment

In [14]:
sentiment_analysis(2015)

{'Negative': 989, 'Neutral': 2873, 'Positive': 2748}